In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)

In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib

In [3]:
import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

ts_info_fullfn = debris_prms.ts_fp + debris_prms.roi + '_debris_tsinfo.nc'

print(ts_info_fullfn)

if os.path.exists(debris_prms.ts_fp) == False:
    os.makedirs(debris_prms.ts_fp)

/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/01_ts_data/01_debris_tsinfo.nc


In [4]:
rgiid_list = []
rgiid_fn_list = []
for i in os.listdir(debris_prms.mb_binned_fp):
    if i.endswith('mb_bins.csv'):
        region = int(i.split('.')[0])
        if region in debris_prms.roi_rgidict[debris_prms.roi]:    
            if region < 10:
                rgiid_list.append(i[0:7])
            else:
                rgiid_list.append(i[0:8])
            rgiid_fn_list.append(i)
        

# rgiid_list = ['18.00718']
# rgiid_fn_list = ['18.00718_mb_bins.csv']

rgiid_list = sorted(rgiid_list)
rgiid_fn_list = sorted(rgiid_fn_list)



main_glac_rgi = debris_prms.selectglaciersrgitable(rgiid_list)
main_glac_rgi['CenLon_360'] = main_glac_rgi['CenLon']
main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + main_glac_rgi.loc[main_glac_rgi['CenLon_360'] < 0, 'CenLon_360'])
main_glac_rgi['bin_fn'] = rgiid_fn_list

1072 glaciers in region 1 are included in this model run: ['00006', '00013', '00027', '00033', '00035', '00037', '00038', '00040', '00041', '00042', '00044', '00045', '00046', '00140', '00148', '00187', '00242', '00312', '00336', '00348', '00351', '00399', '00409', '00426', '00434', '00436', '00537', '00544', '00556', '00557', '00558', '00560', '00561', '00565', '00566', '00569', '00570', '00571', '00572', '00574', '00576', '00578', '00579', '00581', '00582', '00584', '00600', '00660', '00670', '00675'] and more
This study is focusing on 1072 glaciers in region [1]


In [5]:
# Merge with debris cover stats
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])
dc_shp.reset_index(inplace=True, drop=True)

dc_areaperc_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_%'].values))
main_glac_rgi['DC_Area_%'] = main_glac_rgi.RGIId.map(dc_areaperc_dict).fillna(0)
dc_area_dict = dict(zip(dc_shp.RGIId.values,dc_shp['DC_Area_v2'].values))
main_glac_rgi['DC_Area_v2'] = main_glac_rgi.RGIId.map(dc_area_dict).fillna(0)
main_glac_rgi

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float,CenLon_360,bin_fn,DC_Area_%,DC_Area_v2
GlacNo,,,,,,,,,,,,,,,,,,,,,
0,5,RGI60-01.00006,-146.244000,63.571000,1,2,10.470,1201,3547,1740,...,0,9,20090703,6,01.00006,1.00006,213.756000,1.00006_mb_bins.csv,12.11,1194194
1,12,RGI60-01.00013,-146.684082,63.499329,1,2,209.630,823,4003,1848,...,0,3,20090703,13,01.00013,1.00013,213.315918,1.00013_mb_bins.csv,17.84,38937084
2,26,RGI60-01.00027,-146.262817,63.535065,1,2,13.290,1073,2985,1742,...,0,3,20090703,27,01.00027,1.00027,213.737183,1.00027_mb_bins.csv,9.36,1116507
3,32,RGI60-01.00033,-146.872000,63.680000,1,2,4.604,1109,3160,1718,...,0,9,20090703,33,01.00033,1.00033,213.128000,1.00033_mb_bins.csv,20.43,897454
4,34,RGI60-01.00035,-147.442000,63.648000,1,2,36.349,1274,2928,1913,...,0,9,20090703,35,01.00035,1.00035,212.558000,1.00035_mb_bins.csv,3.86,1237798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,26856,RGI60-01.26861,-134.732000,59.454000,1,6,3.545,1404,1879,1506,...,0,9,20050811,26861,01.26861,1.26861,225.268000,1.26861_mb_bins.csv,12.14,369293
1068,27096,RGI60-01.27101,-143.473236,60.709583,1,5,80.219,492,3182,1403,...,0,9,20100919,27101,01.27101,1.27101,216.526764,1.27101_mb_bins.csv,19.29,15504591
1069,27098,RGI60-01.27103,-134.086000,58.943000,1,6,86.656,873,2236,1768,...,0,9,20050811,27103,01.27103,1.27103,225.914000,1.27103_mb_bins.csv,1.72,1312248


In [6]:
# Latitude and longitude index to run the model
#  Longitude must be 0 - 360 degrees
latlon_all = []
for i in os.listdir(debris_prms.ostrem_fp):
    if i.endswith(debris_prms.ostrem_fn_sample.split('XXXX')[1]):
        latlon_fn = i.split(debris_prms.ostrem_fn_sample.split('XXXX')[1])[0]
        # Extract latitude
        lat_str = latlon_fn.split('-')[0]
        if 'N' in lat_str:
            i_lat = int(lat_str.split('N')[0]) / 100
        elif 'S' in lat_str:
            i_lat = -1 * int(lat_str.split('S')[0]) / 100
        # Extract longitude
        lon_str = latlon_fn.split('-')[1]
        i_lon = int(lon_str.split('E')[0]) / 100
        latlon_all.append([i_lat, i_lon, i])
latlon_all = sorted(latlon_all)

lat_all = np.array([x[0] for x in latlon_all])
lon_all = np.array([x[1] for x in latlon_all])
ostrem_fn_all_raw = [x[2] for x in latlon_all]

main_glac_rgi['lat_nearest'] = np.nan
main_glac_rgi['lon_nearest'] = np.nan
main_glac_rgi['ostrem_fn'] = np.nan
for nglac, glac_idx in enumerate(main_glac_rgi.index.values):
    latlon_dist = (((main_glac_rgi.loc[glac_idx,'CenLat'] - lat_all)**2 + 
                    (main_glac_rgi.loc[glac_idx,'CenLon_360'] - lon_all)**2)**0.5)
    latlon_nearidx = np.where(latlon_dist == latlon_dist.min())[0][0]
    
    main_glac_rgi.loc[glac_idx,'lat_nearest'] = lat_all[latlon_nearidx]
    main_glac_rgi.loc[glac_idx,'lon_nearest'] = lon_all[latlon_nearidx]
    main_glac_rgi.loc[glac_idx,'ostrem_fn'] = ostrem_fn_all_raw[latlon_nearidx]
    
ostrem_fn_all = sorted(list(np.unique(main_glac_rgi['ostrem_fn'].values)))
lat_values = np.arange(main_glac_rgi.lat_nearest.min(), main_glac_rgi.lat_nearest.max() + 0.1, 0.25)
lon_values = np.arange(main_glac_rgi.lon_nearest.min(), main_glac_rgi.lon_nearest.max() + 0.1, 0.25)

main_glac_rgi

,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,glacno,rgino_str,RGIId_float,CenLon_360,bin_fn,DC_Area_%,DC_Area_v2,lat_nearest,lon_nearest,ostrem_fn
GlacNo,,,,,,,,,,,,,,,,,,,,,
0,5,RGI60-01.00006,-146.244000,63.571000,1,2,10.470,1201,3547,1740,...,6,01.00006,1.00006,213.756000,1.00006_mb_bins.csv,12.11,1194194,63.50,213.75,6350N-21375E-debris_melt_curve.nc
1,12,RGI60-01.00013,-146.684082,63.499329,1,2,209.630,823,4003,1848,...,13,01.00013,1.00013,213.315918,1.00013_mb_bins.csv,17.84,38937084,63.50,213.25,6350N-21325E-debris_melt_curve.nc
2,26,RGI60-01.00027,-146.262817,63.535065,1,2,13.290,1073,2985,1742,...,27,01.00027,1.00027,213.737183,1.00027_mb_bins.csv,9.36,1116507,63.50,213.75,6350N-21375E-debris_melt_curve.nc
3,32,RGI60-01.00033,-146.872000,63.680000,1,2,4.604,1109,3160,1718,...,33,01.00033,1.00033,213.128000,1.00033_mb_bins.csv,20.43,897454,63.75,213.25,6375N-21325E-debris_melt_curve.nc
4,34,RGI60-01.00035,-147.442000,63.648000,1,2,36.349,1274,2928,1913,...,35,01.00035,1.00035,212.558000,1.00035_mb_bins.csv,3.86,1237798,63.50,212.50,6350N-21250E-debris_melt_curve.nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,26856,RGI60-01.26861,-134.732000,59.454000,1,6,3.545,1404,1879,1506,...,26861,01.26861,1.26861,225.268000,1.26861_mb_bins.csv,12.14,369293,59.50,225.25,5950N-22525E-debris_melt_curve.nc
1068,27096,RGI60-01.27101,-143.473236,60.709583,1,5,80.219,492,3182,1403,...,27101,01.27101,1.27101,216.526764,1.27101_mb_bins.csv,19.29,15504591,60.75,216.50,6075N-21650E-debris_melt_curve.nc
1069,27098,RGI60-01.27103,-134.086000,58.943000,1,6,86.656,873,2236,1768,...,27103,01.27103,1.27103,225.914000,1.27103_mb_bins.csv,1.72,1312248,59.00,226.00,5900N-22600E-debris_melt_curve.nc


In [7]:
# Process each group and derive elevation statistics for the debris cover
year_mean = np.zeros((len(lat_values), len(lon_values)))
year_std = np.zeros((len(lat_values), len(lon_values)))
year_med = np.zeros((len(lat_values), len(lon_values)))
year_mad = np.zeros((len(lat_values), len(lon_values)))
doy_mean = np.zeros((len(lat_values), len(lon_values)))
doy_std = np.zeros((len(lat_values), len(lon_values)))
doy_med = np.zeros((len(lat_values), len(lon_values)))
doy_mad = np.zeros((len(lat_values), len(lon_values)))
dayfrac_mean = np.zeros((len(lat_values), len(lon_values)))
dayfrac_std = np.zeros((len(lat_values), len(lon_values)))
dayfrac_med = np.zeros((len(lat_values), len(lon_values)))
dayfrac_mad = np.zeros((len(lat_values), len(lon_values)))

for nlatlon, ostrem_fn in enumerate(ostrem_fn_all):
# for nlatlon, ostrem_fn in enumerate([ostrem_fn_all[44]]):
    
    main_glac_rgi_subset = main_glac_rgi[main_glac_rgi['ostrem_fn'] == ostrem_fn]
    main_glac_rgi_subset.reset_index(inplace=True, drop=True)
    
    lat_idx = np.where(main_glac_rgi_subset.loc[0,'lat_nearest'] == lat_values)[0][0]
    lon_idx = np.where(main_glac_rgi_subset.loc[0,'lon_nearest'] == lon_values)[0][0]
    
    lat_deg = lat_values[lat_idx]
    lon_deg = lon_values[lon_idx]
    
    print(nlatlon, lat_deg, lon_deg, ostrem_fn)

    doy_list = []
    year_list = []
    dayfrac_list = []
    for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[0]]):
        glac_str = main_glac_rgi_subset.loc[glac_idx,'rgino_str']
        rgiid = main_glac_rgi_subset.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

#         print(nglac, glac_idx, rgiid,'\n')
        
        # Process debris-covered glaciers
        if ((main_glac_rgi_subset.loc[glac_idx, 'DC_Area_%'] > debris_prms.dc_percarea_threshold) | 
            (main_glac_rgi_subset.loc[glac_idx, 'DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold)):

            # Create glacier feature from ice thickness raster
            thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
            thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
            gf = create_glacfeat(thick_dir, thick_fn)

            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if os.path.exists(dc_shp_proj_fn) == False:
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()

            # Add layers
            gf.add_layers(dc_shp_lyr, gf_add_dhdt=False, gf_add_vel=False, gf_add_ts=True, gf_add_ts_info=True)

            # Isolate the debris-covered areas
            gf.ts.mask = gf.dc_mask
            gf.ts_dayfrac.mask = gf.dc_mask
            gf.ts_year.mask = gf.dc_mask
            gf.ts_doy.mask = gf.dc_mask

            # ===== PLOTS =====
#             # DEM
#             var_full2plot = gf.z1.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
#             # Surface temperature
#             var_full2plot = gf.ts.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
#             # Ts Day fraction
#             var_full2plot = gf.ts_dayfrac.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-dayfrac'], 'inferno', 'ts dayfrac', close_fig=False)
#             # Ts Year
#             var_full2plot = gf.ts_year.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-year'], 'inferno', 'ts year', close_fig=False)
#             # Ts Day of year
#             var_full2plot = gf.ts_doy.copy()
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, [glac_str + ' Ts-doy'], 'inferno', 'doy', close_fig=False)
            
        # ========================================================================================================
                    
            doy_list.extend(list(gf.ts_doy.compressed()))
            year_list.extend(list(gf.ts_year.compressed()))
            dayfrac_list.extend(list(gf.ts_dayfrac.compressed()))
    
    # Year
    year_mean_latlon = np.mean(year_list)
    year_std_latlon = np.std(year_list)
    year_med_latlon = malib.fast_median(year_list)
    year_mad_latlon = malib.mad(year_list)
    # Day of year
    #  adjust day of year to avoid issues with comparing JD 1 (Jan 1) and JD 365 (Dec 31),
    #  since the mean would be JD 182 (June), but should be JD 0.5 (Jan 1); also would affect std, etc.
    doy_array = np.array(doy_list)
    doy_array[doy_array > 365] = 365
    doy_array[doy_array > 182] = doy_array[doy_array > 182] - 365
    doy_mean_latlon = np.mean(doy_array)
    if doy_mean_latlon < 0:
        doy_mean_latlon += 365
    doy_std_latlon = np.std(doy_array)
    doy_med_latlon = malib.fast_median(doy_array)
    if doy_med_latlon < 0:
        doy_med_latlon += 365
    doy_mad_latlon = malib.mad(doy_array)
    dayfrac_mean_latlon = np.mean(dayfrac_list)
    dayfrac_std_latlon = np.std(dayfrac_list)
    dayfrac_med_latlon = malib.fast_median(dayfrac_list)
    dayfrac_mad_latlon = malib.mad(dayfrac_list)
    
    # Update array
    year_mean[lat_idx,lon_idx] = year_mean_latlon
    year_std[lat_idx,lon_idx] = year_std_latlon
    year_med[lat_idx,lon_idx] = year_med_latlon
    year_mad[lat_idx,lon_idx] = year_mad_latlon
    doy_mean[lat_idx,lon_idx] = doy_mean_latlon
    doy_std[lat_idx,lon_idx] = doy_std_latlon
    doy_med[lat_idx,lon_idx] = doy_med_latlon
    doy_mad[lat_idx,lon_idx] = doy_mad_latlon
    dayfrac_mean[lat_idx,lon_idx] = dayfrac_mean_latlon
    dayfrac_std[lat_idx,lon_idx] = dayfrac_std_latlon
    dayfrac_med[lat_idx,lon_idx] = dayfrac_med_latlon
    dayfrac_mad[lat_idx,lon_idx] = dayfrac_mad_latlon
    
    print('  year mean +/- std:', np.round(year_mean_latlon,1), np.round(year_std_latlon,1)) 
    print('  doy mean +/- std:', np.round(doy_mean_latlon,1), np.round(doy_std_latlon,1)) 
    print('    doy median +/- mad:', np.round(doy_med_latlon,1), np.round(doy_mad_latlon,1)) 
    print('  dayfrac mean +/- std:', np.round(dayfrac_mean_latlon,3), np.round(dayfrac_std_latlon,3))

0 55.25 230.5 5525N-23050E-debris_melt_curve.nc
  year mean +/- std: 2013.0 0.1
  doy mean +/- std: 225.5 7.1
    doy median +/- mad: 223.8 0.0
  dayfrac mean +/- std: 19.736 0.031
1 55.25 230.75 5525N-23075E-debris_melt_curve.nc
  year mean +/- std: 2013.0 0.0
  doy mean +/- std: 224.7 4.6
    doy median +/- mad: 223.8 0.0
  dayfrac mean +/- std: 19.742 0.018
2 55.75 230.25 5575N-23025E-debris_melt_curve.nc
  year mean +/- std: 2013.3 0.7
  doy mean +/- std: 268.1 107.9
    doy median +/- mad: 223.8 0.0
  dayfrac mean +/- std: 19.737 0.022
3 56.0 229.5 5600N-22950E-debris_melt_curve.nc
  year mean +/- std: 2013.4 0.9
  doy mean +/- std: 251.0 85.5
    doy median +/- mad: 223.8 0.0
  dayfrac mean +/- std: 19.748 0.024
4 56.0 229.75 5600N-22975E-debris_melt_curve.nc
  year mean +/- std: 2013.1 0.5
  doy mean +/- std: 249.6 84.0
    doy median +/- mad: 223.8 0.0
  dayfrac mean +/- std: 19.745 0.027
5 56.0 230.25 5600N-23025E-debris_melt_curve.nc
  year mean +/- std: 2013.1 0.3
  doy mean

/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:107: RuntimeWarning: invalid value encountered in greater
/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/ipykernel_launcher.py:108: RuntimeWarning: invalid value encountered in greater


  year mean +/- std: nan nan
  doy mean +/- std: nan nan
    doy median +/- mad: nan nan
  dayfrac mean +/- std: nan nan
25 57.0 228.0 5700N-22800E-debris_melt_curve.nc
  year mean +/- std: 2013.6 1.2
  doy mean +/- std: 221.4 9.6
    doy median +/- mad: 214.8 0.0
  dayfrac mean +/- std: 19.862 0.049
26 57.0 228.5 5700N-22850E-debris_melt_curve.nc
  year mean +/- std: 2013.1 0.6
  doy mean +/- std: 217.1 9.6
    doy median +/- mad: 214.8 0.0
  dayfrac mean +/- std: 19.857 0.039
27 57.0 228.75 5700N-22875E-debris_melt_curve.nc
  year mean +/- std: 2013.0 0.2
  doy mean +/- std: 218.9 4.6
    doy median +/- mad: 214.8 0.0
  dayfrac mean +/- std: 19.8 0.055
28 57.0 229.0 5700N-22900E-debris_melt_curve.nc
  year mean +/- std: 2013.6 1.0
  doy mean +/- std: 268.8 111.1
    doy median +/- mad: 223.8 0.0
  dayfrac mean +/- std: 19.76 0.05
29 57.0 229.5 5700N-22950E-debris_melt_curve.nc
  year mean +/- std: 2013.0 0.0
  doy mean +/- std: 222.6 3.1
    doy median +/- mad: 223.8 0.0
  dayfrac me

  year mean +/- std: 2014.5 0.9
  doy mean +/- std: 194.8 14.5
    doy median +/- mad: 186.8 0.0
  dayfrac mean +/- std: 19.996 0.027
71 58.25 227.25 5825N-22725E-debris_melt_curve.nc
  year mean +/- std: 2014.2 1.0
  doy mean +/- std: 199.0 14.4
    doy median +/- mad: 186.8 0.0
  dayfrac mean +/- std: 20.003 0.031
72 58.25 227.5 5825N-22750E-debris_melt_curve.nc
  year mean +/- std: 2013.6 0.9
  doy mean +/- std: 207.2 14.0
    doy median +/- mad: 212.8 13.3
  dayfrac mean +/- std: 20.001 0.041
73 58.5 205.5 5850N-20550E-debris_melt_curve.nc
  year mean +/- std: 2014.9 0.5
  doy mean +/- std: 92.8 134.3
    doy median +/- mad: 165.9 0.0
  dayfrac mean +/- std: 21.339 0.016
74 58.5 222.75 5850N-22275E-debris_melt_curve.nc
  year mean +/- std: nan nan
  doy mean +/- std: nan nan
    doy median +/- mad: nan nan
  dayfrac mean +/- std: nan nan
75 58.5 223.0 5850N-22300E-debris_melt_curve.nc
  year mean +/- std: 2013.3 0.9
  doy mean +/- std: 226.3 42.4
    doy median +/- mad: 217.8 0.0
 

  year mean +/- std: 2014.7 0.7
  doy mean +/- std: 216.2 3.3
    doy median +/- mad: 214.9 0.0
  dayfrac mean +/- std: 20.411 0.015
116 59.25 221.75 5925N-22175E-debris_melt_curve.nc
  year mean +/- std: 2014.9 0.5
  doy mean +/- std: 215.5 2.4
    doy median +/- mad: 214.9 0.0
  dayfrac mean +/- std: 20.409 0.01
117 59.25 222.0 5925N-22200E-debris_melt_curve.nc
  year mean +/- std: 2014.8 0.6
  doy mean +/- std: 228.0 60.7
    doy median +/- mad: 214.9 0.0
  dayfrac mean +/- std: 20.402 0.025
118 59.25 222.25 5925N-22225E-debris_melt_curve.nc
  year mean +/- std: 2013.7 0.9
  doy mean +/- std: 222.3 39.1
    doy median +/- mad: 217.8 0.0
  dayfrac mean +/- std: 20.372 0.035
119 59.25 222.75 5925N-22275E-debris_melt_curve.nc
  year mean +/- std: 2013.0 0.0
  doy mean +/- std: 217.9 5.0
    doy median +/- mad: 217.8 0.0
  dayfrac mean +/- std: 20.347 0.004
120 59.25 223.0 5925N-22300E-debris_melt_curve.nc
  year mean +/- std: 2013.0 0.0
  doy mean +/- std: 217.8 0.0
    doy median +/- 

...

  year mean +/- std: 2014.2 0.5
  doy mean +/- std: 209.3 91.0
    doy median +/- mad: 182.9 0.0
  dayfrac mean +/- std: 20.922 0.008
206 60.5 215.5 6050N-21550E-debris_melt_curve.nc
  year mean +/- std: 2015.8 0.6
  doy mean +/- std: 267.7 99.2
    doy median +/- mad: 238.2 0.0
  dayfrac mean +/- std: 20.767 0.081
207 60.5 215.75 6050N-21575E-debris_melt_curve.nc
  year mean +/- std: 2015.3 0.8
  doy mean +/- std: 252.7 109.2
    doy median +/- mad: 235.9 74.1
  dayfrac mean +/- std: 20.829 0.098
208 60.5 216.25 6050N-21625E-debris_melt_curve.nc
  year mean +/- std: 2016.0 0.1
  doy mean +/- std: 238.2 0.5
    doy median +/- mad: 238.2 0.0
  dayfrac mean +/- std: 20.724 0.003
209 60.5 218.0 6050N-21800E-debris_melt_curve.nc
  year mean +/- std: 2015.7 0.6
  doy mean +/- std: 18.7 147.1
    doy median +/- mad: 245.2 13.8
  dayfrac mean +/- std: 20.669 0.056
210 60.5 221.25 6050N-22125E-debris_melt_curve.nc
  year mean +/- std: 2014.4 1.2
  doy mean +/- std: 326.0 148.9
    doy median +

  year mean +/- std: 2015.5 0.5
  doy mean +/- std: 213.9 28.7
    doy median +/- mad: 185.9 0.0
  dayfrac mean +/- std: 20.969 0.057
251 61.0 213.75 6100N-21375E-debris_melt_curve.nc
  year mean +/- std: 2015.4 0.7
  doy mean +/- std: 21.5 166.4
    doy median +/- mad: 169.9 3.7
  dayfrac mean +/- std: 20.936 0.039
252 61.0 214.0 6100N-21400E-debris_melt_curve.nc
  year mean +/- std: 2015.5 0.6
  doy mean +/- std: 294.0 141.0
    doy median +/- mad: 245.2 88.0
  dayfrac mean +/- std: 20.889 0.051
253 61.0 214.25 6100N-21425E-debris_melt_curve.nc
  year mean +/- std: 2015.2 0.4
  doy mean +/- std: 291.4 151.2
    doy median +/- mad: 185.9 0.0
  dayfrac mean +/- std: 20.902 0.038
254 61.0 214.75 6100N-21475E-debris_melt_curve.nc
  year mean +/- std: 2015.2 0.4
  doy mean +/- std: 322.6 160.8
    doy median +/- mad: 245.2 88.0
  dayfrac mean +/- std: 20.895 0.035
255 61.0 215.0 6100N-21500E-debris_melt_curve.nc
  year mean +/- std: 2015.5 0.5
  doy mean +/- std: 309.6 140.5
    doy media

  year mean +/- std: 2014.3 1.0
  doy mean +/- std: 359.8 162.7
    doy median +/- mad: 210.9 26.0
  dayfrac mean +/- std: 21.306 0.048
296 61.5 208.0 6150N-20800E-debris_melt_curve.nc
  year mean +/- std: 2014.0 0.0
  doy mean +/- std: 210.9 0.0
    doy median +/- mad: 210.9 0.0
  dayfrac mean +/- std: 21.333 0.0
297 61.5 211.75 6150N-21175E-debris_melt_curve.nc
  year mean +/- std: 2015.0 0.0
  doy mean +/- std: 358.6 157.7
    doy median +/- mad: 215.9 0.0
  dayfrac mean +/- std: 21.119 0.003
298 61.5 212.0 6150N-21200E-debris_melt_curve.nc
  year mean +/- std: 2014.9 0.3
  doy mean +/- std: 162.3 42.3
    doy median +/- mad: 167.9 0.0
  dayfrac mean +/- std: 21.121 0.027
299 61.5 212.25 6150N-21225E-debris_melt_curve.nc
  year mean +/- std: 2015.0 0.0
  doy mean +/- std: 164.9 30.6
    doy median +/- mad: 167.9 0.0
  dayfrac mean +/- std: 21.116 0.001
300 61.5 212.75 6150N-21275E-debris_melt_curve.nc
  year mean +/- std: nan nan
  doy mean +/- std: nan nan
    doy median +/- mad: n

  year mean +/- std: 2014.4 1.2
  doy mean +/- std: 106.7 127.3
    doy median +/- mad: 169.9 4.4
  dayfrac mean +/- std: 20.847 0.06
341 62.0 217.0 6200N-21700E-debris_melt_curve.nc
  year mean +/- std: 2014.8 0.8
  doy mean +/- std: 121.1 114.9
    doy median +/- mad: 169.9 0.0
  dayfrac mean +/- std: 20.869 0.056
342 62.0 217.25 6200N-21725E-debris_melt_curve.nc
  year mean +/- std: 2014.4 1.5
  doy mean +/- std: 21.4 159.9
    doy median +/- mad: 165.9 10.4
  dayfrac mean +/- std: 20.801 0.038
343 62.0 217.5 6200N-21750E-debris_melt_curve.nc
  year mean +/- std: 2014.6 1.5
  doy mean +/- std: 321.7 151.8
    doy median +/- mad: 213.3 0.0
  dayfrac mean +/- std: 20.799 0.038
344 62.0 217.75 6200N-21775E-debris_melt_curve.nc
  year mean +/- std: 2014.2 1.5
  doy mean +/- std: 338.2 155.4
    doy median +/- mad: 213.9 0.0
  dayfrac mean +/- std: 20.694 0.069
345 62.0 218.5 6200N-21850E-debris_melt_curve.nc
  year mean +/- std: 2014.9 1.4
  doy mean +/- std: 121.5 111.5
    doy median 

  year mean +/- std: 2013.2 0.5
  doy mean +/- std: 200.1 40.2
    doy median +/- mad: 193.9 0.0
  dayfrac mean +/- std: 21.146 0.011
386 63.5 214.25 6350N-21425E-debris_melt_curve.nc
  year mean +/- std: 2015.0 0.2
  doy mean +/- std: 160.7 52.1
    doy median +/- mad: 169.9 0.0
  dayfrac mean +/- std: 20.99 0.099
387 63.5 214.5 6350N-21450E-debris_melt_curve.nc
  year mean +/- std: 2015.0 0.2
  doy mean +/- std: 148.3 81.2
    doy median +/- mad: 169.9 0.0
  dayfrac mean +/- std: 20.955 0.088
388 63.5 214.75 6350N-21475E-debris_melt_curve.nc
  year mean +/- std: 2015.0 0.3
  doy mean +/- std: 154.8 69.7
    doy median +/- mad: 169.9 0.0
  dayfrac mean +/- std: 20.911 0.041
389 63.5 215.25 6350N-21525E-debris_melt_curve.nc
  year mean +/- std: 2015.0 0.0
  doy mean +/- std: 169.9 0.0
    doy median +/- mad: 169.9 0.0
  dayfrac mean +/- std: 20.903 0.0
390 63.75 212.25 6375N-21225E-debris_melt_curve.nc
  year mean +/- std: 2014.1 1.1
  doy mean +/- std: 79.2 149.3
    doy median +/- ma

In [8]:
# Export to dataset
ds_ts_stats = xr.Dataset({'year_mean': (['latitude', 'longitude'], year_mean),
                          'year_std': (['latitude', 'longitude'], year_std),
                          'year_med': (['latitude', 'longitude'], year_med),
                          'year_mad': (['latitude', 'longitude'], year_mad),
                          'doy_mean': (['latitude', 'longitude'], doy_mean),
                          'doy_std': (['latitude', 'longitude'], doy_std),
                          'doy_med': (['latitude', 'longitude'], doy_med),
                          'doy_mad': (['latitude', 'longitude'], doy_mad),
                          'dayfrac_mean': (['latitude', 'longitude'], dayfrac_mean),
                          'dayfrac_std': (['latitude', 'longitude'], dayfrac_std),
                          'dayfrac_med': (['latitude', 'longitude'], dayfrac_med),
                          'dayfrac_mad': (['latitude', 'longitude'], dayfrac_mad),},
                          coords={'latitude': lat_values,
                                  'longitude': lon_values})

attrs_dict={
     'year_mean':{'units':'-',
         'long_name':'mean year',
         'comment': 'mean year when mosaicked surface temperature satellite image was acquired'},
     'year_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of year when mosaicked surface temperature satellite image was acquired'},
     'year_med':{'units':'-',
         'long_name':'median year',
         'comment': 'median year when mosaicked surface temperature satellite image was acquired'},
     'year_mad':{'units':'-',
         'long_name':'median absolute deviation year',
         'comment': 'median absolute deviation of year of when mosaicked surface temperature satellite image was acquired'},
     'doy_mean':{'units':'days since January 1',
         'long_name':'mean day of year',
         'comment': 'mean day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_std':{'units':'days since January 1',
         'long_name':'day of year standard deviation',
         'comment': 'standard deviation of day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_med':{'units':'days since January 1',
         'long_name':'median day of year',
         'comment': 'median day of year when mosaicked surface temperature satellite image was acquired'},
     'doy_mad':{'units':'days since January 1',
         'long_name':'median absolute deviation day of year',
         'comment': 'day of year of year of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mean':{'units':'-',
         'long_name':'mean hour',
         'comment': 'mean hour of when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_std':{'units':'-',
         'long_name':'year standard deviation',
         'comment': 'standard deviation of hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_med':{'units':'-',
         'long_name':'median hour',
         'comment': 'median hour when mosaicked surface temperature satellite image was acquired'},
     'dayfrac_mad':{'units':'-',
         'long_name':'median absolute deviation hour',
         'comment': 'median absolute deviation of hour of when mosaicked surface temperature satellite image was acquired'},}

for vn in ['year_mean', 'year_std', 'year_med', 'year_mad',
           'doy_mean', 'doy_std', 'doy_med', 'doy_mad',
           'dayfrac_mean', 'dayfrac_std', 'dayfrac_med', 'dayfrac_mad',]:
    ds_ts_stats[vn].attrs = attrs_dict[vn]
    
ds_ts_stats.to_netcdf(ts_info_fullfn)
                
print(ds_ts_stats)

<xarray.Dataset>
Dimensions:       (latitude: 35, longitude: 106)
Coordinates:
  * latitude      (latitude) float64 55.25 55.5 55.75 56.0 ... 63.25 63.5 63.75
  * longitude     (longitude) float64 204.5 204.8 205.0 ... 230.2 230.5 230.8
Data variables:
    year_mean     (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_std      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_med      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    year_mad      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_mean      (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_std       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_med       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    doy_mad       (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dayfrac_mean  (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dayfrac_std   (latitude

In [9]:
# lat_idx = 4
# lon_idx = 2
# # lat_idx = 37
# # lon_idx = 46
# print(ds_ts_stats['year_mean'][lat_idx,lon_idx].values, ds_ts_stats['year_std'][lat_idx,lon_idx].values, 
#       ds_ts_stats['doy_mean'][lat_idx,lon_idx].values, ds_ts_stats['doy_std'][lat_idx,lon_idx].values)

In [10]:
ts_info_fullfn

'/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/../output/ts_tif/01_ts_data/01_debris_tsinfo.nc'